# NOTAM + METAR Data Processing

Goals:
- Summarize available NOTAM data from prepared notam_extract file. 
- Select example set of facilities (airports) and obtain METARs for the corresponding facilities and date ranges. 
    + Obtain METARs by accessing API
    + Focus on ceiling and runway visual range. Will require some regular expression processing to extract cloud ceiling altitutde. Take the minimum value of each at each facility and time point. 
- Integrate the two. Will require decisions on temporal resolution.
- Prepare data for visualization


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from os import path

import seaborn as sns
pd.options.display.max_rows = 100

In [2]:
# Read NOTAM extract. Obtain data file from shared Box.com site and place in ~/Documents/Notam_local
data_dir = r'~\Documents\Notam_Local' 

notam = pd.read_excel(path.join(data_dir, 'notam_extract.xlsx'), sheet_name = "data", header = 0)


In [3]:
notam.head()

,notam_location,start_time,end_time,keyword,text
0,KSZY,2019-04-02 01:44:00.0,2019-04-17 00:44:00.0,OBST,OBST TOWER LGT (ASR 1269190) 352126.80N0882231...
1,KCXW,2019-04-02 01:46:00.0,2019-04-17 00:46:00.0,OBST,OBST TOWER LGT (ASR 1212822) 345040.40N0923322...
2,KUOS,2019-04-02 01:49:00.0,2019-04-17 00:49:00.0,OBST,OBST TOWER LGT (ASR 1052889) 351222.50N0854828...
3,M95,2019-04-02 01:50:00.0,2019-04-17 00:50:00.0,OBST,OBST TOWER LGT (ASR 1248915) 334516.20N0875825...
4,2A8,2019-04-02 01:54:00.0,2019-04-17 00:54:00.0,OBST,OBST TOWER LGT (ASR 1280021) 340633.40N0870610...


In [12]:
notam.describe()

,notam_location,start_time,end_time,keyword,text
count,6717,6717,6717,6717,6717
unique,1492,5033,3374,197,2859
top,TJSJ,2019-04-15 09:00:00.0,2019-05-31 19:59:00.0,OBST,AIRSPACE SEE P48 10/062 AEROBATIC ACFT DLY SR-SS
freq,341,18,249,3036,267


In [4]:
# Choose locations with more than 100 NOTAMs
selection = notam.groupby('notam_location').agg({'start_time': 'min', 'end_time': 'max', 'keyword': 'count'})
filtered_selection = selection[(selection.keyword > 100)]
filtered_selection


,start_time,end_time,keyword
notam_location,,,
KDVL,2019-01-10 01:00:00.0,2019-05-31 06:00:00.0,142
KDVT,2018-10-06 09:27:00.0,2019-06-28 22:44:00.0,270
KFMY,2018-09-19 13:31:00.0,2019-05-31 18:00:00.0,256
KHUA,2019-03-15 13:00:00.0,2019-06-07 17:00:00.0,146
KLAX,2019-02-27 17:20:00.0,2019-06-14 19:30:00.0,108
KLCQ,2019-01-02 19:45:00.0,2019-06-03 06:29:00.0,152
KMCO,2018-12-28 10:20:00.0,2019-06-30 18:00:00.0,337
KMOB,2019-03-26 11:20:00.0,2019-06-30 19:00:00.0,128
KMSY,2017-04-26 11:40:00.0,2019-06-30 18:00:00.0,207


In [5]:
# Extract these locations for use in METAR downloading
focal_loc = filtered_selection.index.values
focal_loc[0] # Showing this is now a list 


'KDVL'

In [6]:
starts = filtered_selection.start_time
ends = filtered_selection.end_time

print(type(starts))
starts.describe()


<class 'pandas.core.series.Series'>


count                        12
unique                       12
top       2019-01-02 19:45:00.0
freq                          1
Name: start_time, dtype: object

## Get METARs for these locations and times

- https://mesonet.agron.iastate.edu/request/download.phtml
- https://github.com/akrherz/iem/blob/master/scripts/asos/iem_scraper_example.py

Working steps:
- Adapted functions from `iem_scraper_example.py`, currently in `metar_scraper.py`. Edit in that file, and use `%run metar_scraper` to import the functions into this notebook.
- Changed main function to `get_metar` to use the specific start_time and end_time from filtered_selection data frame
- Compiled in one DataFrame rather than downloading separately by station
- Work on integrating METAR and NOTAM data and visualizing.


In [7]:
# Edit function get_metar in 
# %load metar_scraper 
%run metar_scraper

Provide DataFrame of stations and time range


In [8]:
# Scratch to Test download
#SERVICE = "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"
#service = SERVICE + "data=all&tz=Etc/UTC&format=comma&latlon=yes&"
#startts = dt.datetime.strptime(filtered_selection['start_time'][0], '%Y-%m-%d %H:%M:%S.%f')
#endts = dt.datetime.strptime(filtered_selection['end_time'][0], '%Y-%m-%d %H:%M:%S.%f')

#station_service = service + startts.strftime('year1=%Y&month1=%m&day1=%d&')
#station_service += endts.strftime('year2=%Y&month2=%m&day2=%d&')
#uri = '%s&station=%s' % (station_service, filtered_selection.index[0])
#print('Downloading: %s' % (filtered_selection.index[0], ))
#data = download_data(uri)

#df = pd.DataFrame([x.split(',') for x in data.split('\n')])
#df.columns = df.iloc[5] # Rename the columns with the values in the fifth row
#df = df[6:].reset_index() # Keep only values from the sixth row on, and reset the row index
#print(df)

# View current working directory
#import os
#os.getcwd()

In [11]:
metar_data = get_metar(stations = filtered_selection)

metar_data.to_csv(path.join(data_dir, 'METAR_extract.csv'))


Downloading: KDVL
Downloading: KDVT
Downloading: KFMY
Downloading: KHUA
Downloading: KLAX
Downloading: KLCQ
Downloading: KMCO
Downloading: KMOB
Downloading: KMSY
Downloading: KPHL
Downloading: KTUS
Downloading: TJSJ


In [10]:
metar_data.describe()
metar_data.station.describe()

count     765764
unique        13
top          MSY
freq      248440
Name: station, dtype: object

# Joining METAR and NOTAM data

- `metar_data` is now a DataFrame with all the necessary weather data for all stations.
- We want to first trim this down to just the minimum necessary values (RVR and ceiling) 
- Then we want to join that with the counts of NOTAMs for each station.


In [ ]:
notam_now = notam.loc[notam['notam_location'] == 'KDVL']
notam_now

In [ ]:
#datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')
#datetime_object
#base = datetime.today()
#base
#base

max_end = max(notam_now['end_time'])
min_start = min(notam_now['start_time'])
print(base)
start = dt.datetime.strptime(min_start, "%Y-%m-%d %H:%M:%S.%f")
end = dt.datetime.strptime(max_end, "%Y-%m-%d %H:%M:%S.%f")
print(datetime_object)
difference = (end - start).total_seconds() / 60.0
print(difference)
date_list = [end - dt.timedelta(minutes=x) for x in range(0,int(difference),20)]
print(date_list)

In [ ]:
df2 = pd.DataFrame(date_list[::-1], columns=['Time'])
print(df2)

In [ ]:
df2["count"] = 0
print(df2)

In [ ]:
#df2['count'] = len(notam_now[(notam_now['start_time']<=df2['count']) & (notam_now['end_time']>=df2['count'])])
